In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['titanic', 'forest-cover-type-kernels-only']


In [2]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD as lrSGD
from pyspark.sql import functions as F


In [3]:
spark = SparkSession \
   .builder \
   .appName("Titanic PySpark Exercise") \
   .config("spark.some.config.option", "some-value") \
   .getOrCreate()

## Learning Objectives

- Learn all of the methods in pandas for data-frame manipulation
- The dataset we use is Titanic dataset
- Apply visualization to data-frame

### Lets make Pandas dataframe from titanic csv file 

In [4]:
df = spark.read.csv('../input/titanic/train.csv', header = True)

### Lets look at the first 5 rows of dataframe

In [5]:
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [6]:
df.count(), len(df.columns)

(891, 12)

### Titanic Dataset Description

In [7]:
# VARIABLE DESCRIPTIONS:
# survival        Survival
#                 (0 = No; 1 = Yes)
# pclass          Passenger Class
#                 (1 = 1st; 2 = 2nd; 3 = 3rd)
# name            Name
# sex             Sex
# age             Age
# sibsp           Number of Siblings/Spouses Aboard
# parch           Number of Parents/Children Aboard
# ticket          Ticket Number
# fare            Passenger Fare
# cabin           Cabin
# embarked        Port of Embarkation
#                 (C = Cherbourg; Q = Queenstown; S = Southampton)

### How many of Age values are empty (or null)?

In [8]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



### Create a new column as gender, when Sex is female it is zero when sex is male it is one


In [9]:
df = df.withColumn("Gender", when(df['Sex'] == "female", 0).otherwise(1))
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|     1|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|     0|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|     0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|     0|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|     1|
|          6|       0|  

### We have one more column (check it)

In [10]:
df.count(), len(df.columns)

(891, 13)

### Show the majority of Age range

In [11]:
# GRAPH

### List all of the Ages that are not null

In [43]:
df.where(col("Age").isNotNull()).show()

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|     1|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|     0|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|     0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|     0|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|     1|
|          7|       0|     1|McC

### Slice the dataframe for those whose Embarked section was 'C'

In [13]:
df.filter(df['Embarked'] == "C").show()

+-----------+--------+------+--------------------+------+----+-----+-----+-------------+--------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|       Ticket|    Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+-------------+--------+-----+--------+------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|     PC 17599| 71.2833|  C85|       C|     0|
|         10|       1|     2|Nasser, Mrs. Nich...|female|  14|    1|    0|       237736| 30.0708| null|       C|     0|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|         2649|   7.225| null|       C|     0|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|         2631|   7.225| null|       C|     1|
|         31|       0|     1|Uruchurtu, Don. M...|  male|  40|    0|    0|     PC 17601| 27.7208| null|       C|     1|
|         32|       1|     1|Spencer, Mr

### Plot the Age range for those whose Embraked were 'C'

In [14]:
# GRAPH

### Apply couple of Normal Distributions to Histogram obtained above 

In [15]:
# GRAPH

### Describe a specific column 

In [16]:
df.describe(['Embarked']).show()

+-------+--------+
|summary|Embarked|
+-------+--------+
|  count|     889|
|   mean|    null|
| stddev|    null|
|    min|       C|
|    max|       S|
+-------+--------+



### How many unique values does the 'Embraked' have?

In [17]:
df.select('Embarked').distinct().dropna().count()

3

### Count the different 'Embarked' values the dataframe has

In [18]:
# GRAPH

### Count the different 'Embarked' values the dataframe has and plot horizontaly

In [19]:
# GRAPH

### Another way to do the count and plot it

In [20]:
# GRAPH

In [21]:
# df.select('Embarked').distinct().count()

df.groupBy("Embarked").count().dropna().show()
# .count().orderBy().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  644|
+--------+-----+



In [22]:
df.groupBy("Sex").count().toJSON().collect()

['{"Sex":"female","count":314}', '{"Sex":"male","count":577}']

In [23]:
# GRAPH

In [24]:
# GRAPH

### Plot how many of the passengers were children, youth, middle age and old based on there Sex for those who 'Embarked' in section 'C'?

In [81]:
# TODO
# for i in df[df['Embarked'] == 'C'].groupby('Sex')['Age']:
#     print(i)
# df.groupBy("Embarked").agg(df['Embarked'] == 'C').count()

# df.filter(df['Embarked'] == "C").dropna().show()

[Row(Embarked='C', Sex='female', avg(Age)=35.35294117647059),
 Row(Embarked='C', Sex='male', avg(Age)=39.774193548387096),
 Row(Embarked='Q', Sex='female', avg(Age)=33.0),
 Row(Embarked='Q', Sex='male', avg(Age)=44.0),
 Row(Embarked='S', Sex='female', avg(Age)=30.952830188679247),
 Row(Embarked='S', Sex='male', avg(Age)=37.71301587301588)]

In [26]:
# GRAPH

In [27]:
# GRAPH

### What is the average Age for female and male (based on sex) for those who have 'Embarked' on section 'C'?

In [82]:
sorted(df.dropna().groupBy('Embarked', 'Sex').agg({'Age': 'mean'}).collect())

[Row(Embarked='C', Sex='female', avg(Age)=35.35294117647059),
 Row(Embarked='C', Sex='male', avg(Age)=39.774193548387096),
 Row(Embarked='Q', Sex='female', avg(Age)=33.0),
 Row(Embarked='Q', Sex='male', avg(Age)=44.0),
 Row(Embarked='S', Sex='female', avg(Age)=30.952830188679247),
 Row(Embarked='S', Sex='male', avg(Age)=37.71301587301588)]

### Another way we can do the above task

In [38]:
# Same as above

[Row(Sex='female', count=314), Row(Sex='male', count=577)]

### Which Age is the oldest for female and male (based on sex) for those who have 'Embarked' on section 'C'?

In [83]:
sorted(df.dropna().groupBy('Embarked', 'Sex').agg({'Age': 'max'}).collect())

[Row(Embarked='C', Sex='female', max(Age)='60'),
 Row(Embarked='C', Sex='male', max(Age)='71'),
 Row(Embarked='Q', Sex='female', max(Age)='33'),
 Row(Embarked='Q', Sex='male', max(Age)='44'),
 Row(Embarked='S', Sex='female', max(Age)='63'),
 Row(Embarked='S', Sex='male', max(Age)='80')]

### For different Ages, plot the Fare they have paid?

In [ ]:
# GRAPH

In [ ]:
# GRAPH

### Plot how percentage Survived for two Sex group based on the passengers class 

In [ ]:
# GRAPH

### Plot how many male or female were in different Passenger classes

In [ ]:
# GRAPH

In [ ]:
# GRAPH

In [103]:
df.crosstab("Sex", "Survived").toJSON().collect()

['{"Sex_Survived":"male","0":468,"1":109}',
 '{"Sex_Survived":"female","0":81,"1":233}']

### Verify values obtained for pertentage 

In [95]:
sorted(df.groupBy("Sex", "Survived", "Pclass").count().toJSON().collect())

['{"Sex":"female","Survived":"0","Pclass":"1","count":3}',
 '{"Sex":"female","Survived":"0","Pclass":"2","count":6}',
 '{"Sex":"female","Survived":"0","Pclass":"3","count":72}',
 '{"Sex":"female","Survived":"1","Pclass":"1","count":91}',
 '{"Sex":"female","Survived":"1","Pclass":"2","count":70}',
 '{"Sex":"female","Survived":"1","Pclass":"3","count":72}',
 '{"Sex":"male","Survived":"0","Pclass":"1","count":77}',
 '{"Sex":"male","Survived":"0","Pclass":"2","count":91}',
 '{"Sex":"male","Survived":"0","Pclass":"3","count":300}',
 '{"Sex":"male","Survived":"1","Pclass":"1","count":45}',
 '{"Sex":"male","Survived":"1","Pclass":"2","count":17}',
 '{"Sex":"male","Survived":"1","Pclass":"3","count":47}']

In [90]:
91/(91 + 3)

0.9680851063829787

In [ ]:
# Same as above

### Stack plot of count based on Sex for different Passenger Class

In [ ]:
# GRAPH

### Stack plot of count based on Sex and Survival for different Passenger Class

In [ ]:
# GRAPH

### Sometimes it is hard to read values from plot, what are the number of female and male at each Passenger Class

In [104]:
df.crosstab("Sex", "Pclass").show()

+----------+---+---+---+
|Sex_Pclass|  1|  2|  3|
+----------+---+---+---+
|      male|122|108|347|
|    female| 94| 76|144|
+----------+---+---+---+



In [105]:
df.crosstab("Sex", "Survived").show()

+------------+---+---+
|Sex_Survived|  0|  1|
+------------+---+---+
|        male|468|109|
|      female| 81|233|
+------------+---+---+



In [107]:
df.dropna().crosstab("Sex", "Embarked").show()

+------------+---+---+---+
|Sex_Embarked|  C|  Q|  S|
+------------+---+---+---+
|        male| 31|  1| 63|
|      female| 34|  1| 53|
+------------+---+---+---+



### How to represent the above cross tab in percentage and graphically present 

In [108]:
# GRAPH

## Question:

What percent of passengers embarked at C?

In [111]:
# Answer:

df.groupBy("Embarked").count().dropna().show()

df.filter(df['Embarked'] == "C").count()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  644|
+--------+-----+



168

In [140]:
#TODO
# sum(dict(df['Embarked'].value_counts()).values())
# df['Embarked'].dropna().count()

df.select(sum(when(col("col_1") === "yes", 1)).as("count")).show(false)

df.select(sum(when(col("Embarked")., 1)).as("count")).show(false)


SyntaxError: invalid syntax (<ipython-input-140-81d11f341e90>, line 5)

In [124]:
# TODO
df.filter(df['Embarked'] == "C").count()/df.count()

0.18855218855218855

What percent of female passengers embarked at C?

In [127]:
df.dropna().crosstab('Sex', 'Embarked').show()

+------------+---+---+---+
|Sex_Embarked|  C|  Q|  S|
+------------+---+---+---+
|        male| 31|  1| 63|
|      female| 34|  1| 53|
+------------+---+---+---+



In [130]:
df.filter((df['Sex'] == 'female') & (df['Embarked'] == "C")).count()

73

In [131]:
df.filter((df['Sex'] == 'female')).count()

314

In [132]:
73/314

0.23248407643312102

In [133]:
df.filter((df['Sex'] == 'female') & (df['Embarked'] == "C")).count()/df.filter((df['Sex'] == 'female')).count()

0.23248407643312102